#### El cliente ha pedido realizar un ajuste a las reglas implementadas hasta ahora, estos ajustes consisten en reemplazar algunas funciones hechas con groupBy por funciones Window, presta mucha atención y resuelve las siguientes actividades.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de las sesiones 04, 05

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.py

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession, DataFrame, Column, Window
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_8") \
    .getOrCreate()

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["04/movies", "04/ratings", "04/tags"]
df_dict = read_tmp_df(spark, names_list)

movies_df = df_dict["04/movies"]
ratings_df = df_dict["04/ratings"]
tags_df = df_dict["04/tags"]

movies_df.show(1, False)
ratings_df.show(1, False)
tags_df.show(1, False)

#### Actividad 1:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 2 de la sesión 05 - método calculate_rating_values" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### def calculate_rating_values_w(df: DataFrame) -> DataFrame:

In [ ]:
## TU CODIGO VA EN ESTA CELDA:

def calculate_rating_values_w(df: DataFrame) -> DataFrame:
    return df # rating_df modificar codigo interno

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
calculate_rating_values_w(ratings_df).show(2)
"""
Ejemplo de salida esperada:
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|
|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows
"""

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
rating_values_df = calculate_rating_values_w(ratings_df)

assert type(rating_values_df) == DataFrame
assert len(rating_values_df.columns) == 5
assert rating_values_df.count() == 83239

data = rating_values_df \
    .filter(f.col("movie_id") == "296") \
    .collect()[0]

assert data["movie_id"] == "296"
assert data["avg_rating"] == 4.19
assert data["stddev_rating"] == 0.95
assert data["count_rating"] == 108756
assert data["min_time_rating"] == datetime.datetime(1996, 2, 29, 10, 48, 44)

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(rating_values_df, "09/ratings")]

write_tmp_df(dfs)

#### Actividad 2:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 3 de la sesión 05 - métodos get_act_3_df1 y get_act_3_df2" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### Primer tabla (utilizando función concat_ws o concat): def get_act_3_df1_w(df: DataFrame) -> DataFrame:
- ##### Segunda tabla (utilizando función struct): def get_act_3_df2_w(df: DataFrame) -> DataFrame:

In [ ]:
## TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES NUEVAS SI ASI LO REQUIERES

def get_act_3_df1_w(df: DataFrame) -> DataFrame:
    return df # ... transformaciones a tags_df

def get_act_3_df2_w(df: DataFrame) -> DataFrame:
    return df # ... transformaciones a tags_df

act_3_df1 = get_act_3_df1_w(tags_df)
act_3_df2 = get_act_3_df2_w(tags_df)

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
act_3_df1.show(2, False)
act_3_df2.show(2, False)
"""
Ejemplo de salida esperada:
+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |2018-05-26 16:40:54|
|100070  |[{COMEDIAN, 2}, {COMEDY, 1}, {GOOD HUMOUR, 1}, {STRUGGLING CAREER, 1}]|2017-05-19 17:17:36|
+--------+----------------------------------------------------------------------+-------------------+
only showing top 2 rows
"""

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
expected_df1 = [
    Row(movie_id='100070',
        tag_count=['COMEDIAN : 2', 'COMEDY : 1', 'GOOD HUMOUR : 1', 'STRUGGLING CAREER : 1'],
        min_time_tag=datetime.datetime(2017, 5, 19, 17, 17, 36))]
schema_df1 = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("tag_count", t.ArrayType(t.StringType())),
    t.StructField("min_time_tag", t.TimestampType())
])
assert len(act_3_df1.columns) == 3
assert "movie_id" in act_3_df1.columns
assert "tag_count" in act_3_df1.columns
assert "min_time_tag" in act_3_df1.columns
assert schema_to_ddl(spark, act_3_df1.select("movie_id", "tag_count", "min_time_tag")).replace(" NOT NULL", "") == 'movie_id STRING,tag_count ARRAY<STRING>,min_time_tag TIMESTAMP'
assert act_3_df1.count() == 53452
assert act_3_df1.filter(f.col("movie_id") == "100070").subtract(spark.createDataFrame(expected_df1, schema_df1)).count() == 0


expected_df2 = [
    Row(movie_id='100070',
        tag_count=[Row(tag='COMEDIAN', count=2),
                   Row(tag='COMEDY', count=1),
                   Row(tag='GOOD HUMOUR', count=1),
                   Row(tag='STRUGGLING CAREER', count=1)],
        min_time_tag=datetime.datetime(2017, 5, 19, 17, 17, 36))]
schema_df2 = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("tag_count", t.ArrayType(t.StructType([
        t.StructField("tag", t.StringType()),
        t.StructField("count", t.LongType())
    ]))),
    t.StructField("min_time_tag", t.TimestampType())
])
assert len(act_3_df2.columns) == 3
assert "movie_id" in act_3_df2.columns
assert "tag_count" in act_3_df2.columns
assert "min_time_tag" in act_3_df2.columns
assert schema_to_ddl(spark, act_3_df2.select("movie_id", "tag_count", "min_time_tag")).replace(" NOT NULL", "") == 'movie_id STRING,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>>,min_time_tag TIMESTAMP'
assert act_3_df2.count() == 53452
assert act_3_df2.filter(f.col("movie_id") == "100070").subtract(spark.createDataFrame(expected_df2, schema_df2)).count() == 0

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(act_3_df1, "07/tags_p1"),
       (act_3_df2, "07/tags_p2")]

write_tmp_df(dfs)

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
last_movies_df = movies_df \
    .join(act_3_df2, ["movie_id"], "left") \
    .join(rating_values_df, ["movie_id"], "left")

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(last_movies_df, "07/movies")]

write_tmp_df(dfs)

#### Actividad 3:
##### TO DO ->    La última actividad a realizar es la siguiente:
- ##### El cliente nos ha solicitado generar una tabla donde se muestre el top de peliculas ranqueadas por genero, para realizar esto se realizan los siguientes pasos:
    - ##### Las peliculas que entran a la parte del ranqueo deben cumplir con las siguientes condiciones:
        -  count_rating > 1000
        -  avg_rating >= 4.2
        -  stddev_rating < 2
    - #####  Descomponer la columna "genres" en una columna llamada "genre"
    - ##### Agregar la columna "top" donde se asignará el valor de la función "rank" de Spark tomando las siguientes caracteristicas:
        - particionar por: "genre"
        - order por: avg_rating DESC, stddev_rating ASC, count_rating DESC
##### El esquema resultante deberá ser:
* |-- title: string
* |-- year: integer
* |-- genre: string
* |-- top: integer

In [ ]:
## TU CODIGO VA EN ESTA CELDA:

top_movies_df = last_movies_df # Transformaciones a last_movies_df

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

top_movies_df.show(5)
"""
Ejemplo de salida esperada (hay que mantener mismo orden de columnas):
+--------------------+----+---------+---+
|               title|year|    genre|top|
+--------------------+----+---------+---+
|Band of Brothers ...|2001|   Action|  1|
|Seven Samurai (Sh...|1954|   Action|  2|
|   Fight Club (1999)|1999|   Action|  3|
|Over the Garden W...|2013|Adventure|  1|
|Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows
"""

In [ ]:
assert top_movies_df.orderBy(f.col("genre").asc(), f.col("top").asc()).collect() == top_movies

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(top_movies_df, "07/top_movies")]

write_tmp_df(dfs)